In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.model_selection import train_test_split


sns.set_style('white')

In [2]:
# select closed loans
loans = pd.read_csv('../data/loan.csv')
loans['grade'] = loans['grade'].astype('category', ordered=True)
loans['last_pymnt_d'] = pd.to_datetime(loans['last_pymnt_d'])
closed_status = ['Fully Paid', 'Charged Off',
              'Does not meet the credit policy. Status:Fully Paid',
              'Does not meet the credit policy. Status:Charged Off']
closed_loans = loans[loans['loan_status'].isin(closed_status)]
paid_status = ['Fully Paid', 'Does not meet the credit policy. Status:Fully Paid']
closed_loans['paid'] = [True if loan in paid_status else False for loan in closed_loans['loan_status']]
closed_loans.shape

/Users/ro.d.bruijn/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ro.d.bruijn/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(255720, 75)

In [3]:
# logistic regression, GBM, Random Forest
# train (60%), val (20%), test (20%) adviced by Andrew Ng

In [4]:
# make a separate testset
X_trainval, X_test, y_trainval, y_test = train_test_split(closed_loans.iloc[:,0:74], 
                                                    closed_loans['paid'], test_size=0.5, 
                                                    random_state=123, stratify=closed_loans['loan_status'])

In [ ]:
# cross_val_score
# f1_weighted
# StratifiedKFold: stratified? for loan_status/paid? --> predefined splits?

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
scores = cross_val_score(clf, X_trainval, y_trainval, cv=5)

ValueError: could not convert string to float: 'Source Verified'

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
scores = cross_val_score(clf, X_trainval, y_trainval, cv=5)

ValueError: could not convert string to float: 'INDIVIDUAL'